version-1, 30-Nov ,  ( after 7:30 pm)

# **Finetuning ```Llama-3.2-3B-Instruct``` model using ```LORA``` on  ```SAMSum dataset``` (abstractive dialogue summaries) for summarization task**


*   **Author:** ```Pratik Vyas```
*   **Task:** ```Summarization```
*   **Base Model from which model finetuned:** ```meta-llama/Llama-3.2-3B-Instruct``` ( https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct )
*   **Dataset:** ```SAMSum``` ( https://paperswithcode.com/dataset/samsum-corpus )







# **Import Libs**

In [ ]:
!pip3 install -q -U accelerate
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers
!pip install -q rouge_score
!pip install -q optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 16.4 MB/s eta 0:00:00
ER

In [ ]:
import torch

print("Is CUDA available? ", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device name: ", torch.cuda.get_device_name(0))

Is CUDA available?  True
Device name:  Tesla T4


In [ ]:
from peft import LoraConfig
from datasets import load_dataset
from datasets import load_metric
import pandas as pd
import numpy as np

import transformers
from trl import SFTTrainer
from rouge_score import rouge_scorer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata


In [ ]:
import os

os.environ["HF_TOKEN"] = "YOUR_HUGGINGFACE_KEY"
os.environ["WEIGHT_BIASES"] = "YOUR_WEIGHTS_BIAS_KEY"

# **Load Model and tokenizer**

In [ ]:
# load a pre-trained tokenizer from the Hugging Face Model Hub, with authentication for the Hugging Face API token

model_id = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HF_TOKEN"])

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# The BitsAndBytesConfig configuration is used to specify settings for quantizing a model to use 4-bit precision,
# which can help reduce the model's memory footprint and improve inference speed
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# load a pre-trained causal language model with specific quantization settings and automatic device mapping.
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
from IPython.display import Markdown


content = (
    "Gemma is a family of lightweight, state-of-the-art open models built from the same research and technology \
          used to create the Gemini models. Developed by Google DeepMind and other teams across Google, \
          Gemma is named after the Latin gemma, meaning precious stone."
)

prompt = f"User:Summarize below in one sentence: \n\n {content} \n\n AI Summary:\n"


device = "cuda:0"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)

Markdown(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


User:Summarize below in one sentence: 

 Gemma is a family of lightweight, state-of-the-art open models built from the same research and technology           used to create the Gemini models. Developed by Google DeepMind and other teams across Google,           Gemma is named after the Latin gemma, meaning precious stone. 

 AI Summary:
 Gemma is a family of lightweight, state-of-the-art open models built from the same research and technology used to create the Gemini models. Developed by Google DeepMind and other teams across Google, Gemma is named after the Latin gemma, meaning

# **LORA-Finetuning**

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e

In [ ]:
import bitsandbytes as bnb


#  identifies all the linear layers in a model
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if "lm_head" in lora_module_names:
        lora_module_names.remove("lm_head")
    return list(lora_module_names)


modules = find_all_linear_names(model)

print(modules)

['v_proj', 'gate_proj', 'o_proj', 'k_proj', 'down_proj', 'q_proj', 'up_proj']


## **Load Dataset**

In [ ]:
from datasets import load_dataset


## list of dataset for summarization. Choose one of them for your task
# https://paperswithcode.com/dataset/cnn-daily-mail-1
# data = load_dataset("knkarthick/dialogsum") ##Dialogue Summarization Dataset
# data = load_dataset("cnn_dailymail","3.0.0")
# data = load_dataset("GEM/wiki_lingua")

!pip install -q py7zr
data = load_dataset("samsum")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 28.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
data["train"][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [ ]:
from datasets import DatasetDict

dataset_dict = DatasetDict(data)

DATA_RECORD_SIZE = 10  # size of train/test dataset

# Extract the first 100 rows from the training dataset
training_dataset = dataset_dict["train"].select(range(DATA_RECORD_SIZE))

# Extract the first 100 rows from the training dataset
val_dataset = dataset_dict["validation"].select(range(DATA_RECORD_SIZE))

print(training_dataset)
print(val_dataset)

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 10
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 10
})


In [ ]:
!pip3 install -q -U wandb

In [ ]:
# integrate Weights & Biases (W&B) with training process for tracking, monitoring, and collaboration

import wandb

wandb.login(key=os.environ["WEIGHT_BIASES"])
run = wandb.init(
    project="Llama-3.2-3B-Instruct_FineTuning", job_type="training", anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pratik_ai. Use `wandb login --relogin` to force relogin


In [ ]:
def create_prompt(example):
    text = f"user:\nSummarise dialogue in one sentence: {example['dialogue']} {example['summary']}"
    return [text]

## **LORA hyper-parameters tuning with optuna**

In [ ]:
import optuna

# NUM_OF_EPOCHS = 10
NUM_OF_ITERATION = 20  # this param override NUM_OF_EPOCHS


def objective(trial):
    # Define hyperparameters to tune
    learning_rate = trial.suggest_loguniform("learning_rate", 2e-4, 3e-4)
    lora_dropout = trial.suggest_uniform("lora_dropout", 0.01, 0.03)
    lora_r = trial.suggest_int("lora_r", 2, 4)
    lora_alpha = trial.suggest_int("lora_alpha", 16, 64)
    optim = trial.suggest_categorical(
        "optim", ["paged_adamw_8bit", "paged_adamw_32bit"]
    )
    target_modules = trial.suggest_categorical(
        "target_modules",
        [
            ["q_proj", "v_proj"],
            ["q_proj", "k_proj", "v_proj"],
            # ["k_proj", "v_proj"],
            [
                "q_proj",
                "o_proj",
                "k_proj",
                "v_proj",
                "gate_proj",
                "up_proj",
                "down_proj",
            ],
        ],
    )

    gradient_accumulation_steps = trial.suggest_int("gradient_accumulation_steps", 2, 3)

    lora_config = LoraConfig(
        r=lora_r,  # hyperparam tuning
        lora_alpha=lora_alpha,  # hyperparam tuning
        lora_dropout=lora_dropout,  # hyperparam tuning
        target_modules=target_modules,  # hyperparam tuning
        task_type="CAUSAL_LM",
    )

    # Define training arguments
    training_arguments = transformers.TrainingArguments(
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=gradient_accumulation_steps,  # hyperparam tuning
        # num_train_epochs=NUM_OF_EPOCHS,
        warmup_steps=10,
        eval_strategy="steps",
        eval_steps=0.2,
        max_steps=NUM_OF_ITERATION,
        learning_rate=learning_rate,  # hyperparam tuning
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim=optim,  # hyperparam tuning
        report_to="wandb",  # set none to disable
    )

    # Initialize Trainer
    tokenizer.pad_token = tokenizer.eos_token  # Ensure pad token is set
    # Mistral-7B model, especially since it is a decoder-only model, it is generally recommended to set padding_side to "left".
    tokenizer.padding_side = "left"
    trainer = SFTTrainer(
        model=model,
        train_dataset=training_dataset,
        eval_dataset=val_dataset,
        max_seq_length=512,  # max seq length to input/output. It is crucial for GPU memory management
        args=training_arguments,
        peft_config=lora_config,
        formatting_func=create_prompt,  # preprocessing function before input
        processing_class=tokenizer,
    )

    # Train the model
    model.config.use_cache = False
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    return eval_results["eval_loss"]


# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print("\n\nBest hyperparameters: \n", study.best_params)

[I 2024-11-30 13:57:00,037] A new study created in memory with name: no-name-339b9133-a7ed-4cfd-b358-44d74ff6f5d8
<ipython-input-18-4a0d5486aff3>:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate',  2e-4,3e-4)
<ipython-input-18-4a0d5486aff3>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  lora_dropout = trial.suggest_uniform('lora_dropout', 0.01,0.03)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains ['q_proj', 'v_proj'] which is of type list.
  warnings.warn(message)
/usr/l

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
4,2.781600,2.884114
8,2.695700,2.823886
12,2.331300,2.694522
16,1.695800,2.731432
20,1.030000,2.856431


...

Step,Training Loss,Validation Loss
4,0.017400,3.259018
8,0.013800,3.328955
12,0.000500,3.406066
16,0.000000,3.468433
20,0.000000,3.483355


[I 2024-11-30 14:04:20,008] Trial 18 finished with value: 3.4833552837371826 and parameters: {'learning_rate': 0.00022307097313075983, 'lora_dropout': 0.013717912474788369, 'lora_r': 2, 'lora_alpha': 30, 'optim': 'paged_adamw_32bit', 'target_modules': ['q_proj', 'k_proj', 'v_proj'], 'gradient_accumulation_steps': 3}. Best is trial 10 with value: 2.683363676071167.
<ipython-input-18-4a0d5486aff3>:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate',  2e-4,3e-4)
<ipython-input-18-4a0d5486aff3>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  lora_dropout = trial.suggest_uniform('lora_dropout', 0.01,0.03)
/usr/local/lib/python3.10/dist-

Step,Training Loss,Validation Loss
4,2.781600,2.872041
8,2.419600,2.718901
12,1.675100,2.678894
16,0.517800,2.928895
20,0.123300,3.092626


[I 2024-11-30 14:04:45,161] Trial 19 finished with value: 3.0926263332366943 and parameters: {'learning_rate': 0.00025692743074224976, 'lora_dropout': 0.017304903541012457, 'lora_r': 3, 'lora_alpha': 21, 'optim': 'paged_adamw_8bit', 'target_modules': ['q_proj', 'o_proj', 'k_proj', 'v_proj', 'gate_proj', 'up_proj', 'down_proj'], 'gradient_accumulation_steps': 3}. Best is trial 10 with value: 2.683363676071167.




Best hyperparameters: 
 {'learning_rate': 0.00020313966369710248, 'lora_dropout': 0.015935074164257663, 'lora_r': 3, 'lora_alpha': 18, 'optim': 'paged_adamw_8bit', 'target_modules': ['q_proj', 'k_proj', 'v_proj'], 'gradient_accumulation_steps': 3}


**Retrieve the best hyperparameters**

In [ ]:
# Retrieve the best hyperparameters
best_params = study.best_params
print("Best hyperparameters: \n", best_params)

Best hyperparameters: 
 {'learning_rate': 0.00020313966369710248, 'lora_dropout': 0.015935074164257663, 'lora_r': 3, 'lora_alpha': 18, 'optim': 'paged_adamw_8bit', 'target_modules': ['q_proj', 'k_proj', 'v_proj'], 'gradient_accumulation_steps': 3}


## **Train final model with best hyperparameters**

In [ ]:
# #Load model for training

# Clear GPU cache before loading the model for the second time
torch.cuda.empty_cache()

# Load model for training with CPU offloading enabled
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    # Enable CPU offloading for specific layers
    llm_int8_enable_fp32_cpu_offload=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",  # Let Transformers automatically decide device placement
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
DATA_RECORD_SIZE = 100  # size of training dataset

# Extract the first 100 rows from the training dataset
training_dataset = dataset_dict["train"].select(range(DATA_RECORD_SIZE))

# Extract the first 100 rows from the training dataset
val_dataset = dataset_dict["validation"].select(range(DATA_RECORD_SIZE))

print(training_dataset)
print(val_dataset)

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 100
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 100
})


In [ ]:
# Use the best hyperparameters to train the final model
best_lora_dropout = best_params["lora_dropout"]
best_lora_r = best_params["lora_r"]
best_lora_alpha = best_params["lora_alpha"]
best_optim = best_params["optim"]
best_target_modules = best_params["target_modules"]
best_learning_rate = best_params["learning_rate"]
best_gradient_accumulation_steps = best_params["gradient_accumulation_steps"]


## Training Arguments Parameters

1. **`per_device_train_batch_size`**:
   - **Description**: The batch size per device (GPU/TPU/CPU) during training.
   - **Example**: If you have 2 GPUs and set `per_device_train_batch_size=1`, the effective batch size will be 2.

2. **`per_device_eval_batch_size`**:
   - **Description**: The batch size per device (GPU/TPU/CPU) during evaluation.
   - **Example**: If you have 2 GPUs and set `per_device_eval_batch_size=1`, the effective batch size will be 2.

3. **`gradient_accumulation_steps`**:
   - **Description**: The number of steps to accumulate gradients before performing a backward/update pass.
   - **Example**: If set to 2, the model will accumulate gradients over 2 steps before updating the model parameters, effectively doubling the batch size.

4. **`num_train_epochs`**:
   - **Description**: The total number of training epochs.
   - **Example**: If set to 1, the model will see the entire training dataset once.

5. **`warmup_steps`**:
   - **Description**: The number of steps to perform learning rate warmup.
   - **Example**: If set to 2, the learning rate will gradually increase over the first 2 steps.

6. **`eval_strategy`**:
   - **Description**: The evaluation strategy to use during training.
   - **Options**: `"no"` (no evaluation), `"steps"` (evaluate every `eval_steps`), `"epoch"` (evaluate at the end of each epoch).
   - **Example**: If set to `"steps"`, the model will be evaluated every `eval_steps`.

7. **`eval_steps`**:
   - **Description**: The number of steps between evaluations.
   - **Example**: If set to 0.2, the model will be evaluated every 0.2 steps. Note that this is an unusual setting; typically, `eval_steps` is an integer.

8. **`max_steps`**:
   - **Description**: The total number of training steps to perform.
   - **Example**: If set to 50, the training will stop after 50 steps, regardless of the number of epochs.

9. **`learning_rate`**:
   - **Description**: The initial learning rate for the optimizer.
   - **Example**: This is a hyperparameter that can be tuned. In this context, it is set dynamically based on the value suggested by Optuna.

10. **`fp16`**:
    - **Description**: Whether to use 16-bit (mixed) precision training instead of 32-bit.
    - **Example**: If set to `True`, the model will use mixed precision training, which can speed up training and reduce memory usage.

11. **`logging_steps`**:
    - **Description**: The number of steps between logging events.
    - **Example**: If set to 1, the model will log training metrics every step.

12. **`output_dir`**:
    - **Description**: The directory where the model checkpoints and other outputs will be saved.
    - **Example**: If set to `"outputs"`, all outputs will be saved in the `outputs` directory.

13. **`optim`**:
    - **Description**: The optimizer to use.
    - **Options**: `"adamw_torch"`, `"paged_adamw_32bit"`, `"adamw_hf"`, etc.
    - **Example**: This is a hyperparameter that can be tuned. In this context, it is set dynamically based on the value suggested by Optuna.

14. **`report_to`**:
    - **Description**: The list of integrations to report the results and logs to.
    - **Options**: `"none"`, `"wandb"`, `"tensorboard"`, etc.
    - **Example**: If set to `"none"`, no reporting will be done to any integration.

In [ ]:
SAVE_MODEL = True
# NUM_OF_EPOCHS = 10
NUM_OF_ITERATION = 20  # this param override NUM_OF_EPOCHS


# Define LoRA configuration with the best hyperparameters
lora_config = LoraConfig(
    r=best_lora_r,
    lora_alpha=best_lora_alpha,
    lora_dropout=best_lora_dropout,
    target_modules=best_target_modules,
    task_type="CAUSAL_LM",
)

# Define training arguments with the best hyperparameters
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=best_gradient_accumulation_steps,  # hyperparam tuning
    # num_train_epochs=NUM_OF_EPOCHS,
    warmup_steps=2,
    eval_strategy="steps",  # "epoch", "steps",
    eval_steps=0.2,
    max_steps=NUM_OF_ITERATION,
    learning_rate=best_learning_rate,  # hyperparam tuning
    fp16=True,
    logging_steps=1,
    output_dir="final_outputs",
    optim=best_optim,  # hyperparam tuning
    report_to="wandb",
)


# Initialize Trainer with the best hyperparameters
tokenizer.pad_token = tokenizer.eos_token  # Ensure pad token is set
tokenizer.padding_side = (
    "left"  # decoder-only model, it is recommended to set padding_side to "left".
)

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    eval_dataset=val_dataset,
    peft_config=lora_config,
    max_seq_length=512,  ## ## max length to input/output. It is crucial for GPU memory management
    dataset_text_field="dialogue",
    formatting_func=create_prompt,  # preprocessing function before input
    processing_class=tokenizer,
    args=training_arguments,
    packing=False,  # The trainer will attempt to pack multiple sequences into a single batch to maximize the utilization of the available sequence length,
)


# Train the final model
model.config.use_cache = False
trainer.train()

# Evaluate the final model
final_eval_results = trainer.evaluate()
print("\n--->Final evaluation results:\n", final_eval_results)

# Save the final model and tokenizer
if SAVE_MODEL is True:
    model.save_pretrained("final_model")
    tokenizer.save_pretrained("final_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss
4,2.781500,2.872476
8,2.543400,2.761500
12,2.344600,2.709419
16,2.171500,2.695901
20,2.049300,2.695869



--->Final evaluation results:
 {'eval_loss': 2.695868730545044, 'eval_runtime': 0.4648, 'eval_samples_per_second': 2.151, 'eval_steps_per_second': 2.151, 'epoch': 20.0}


('final_model/tokenizer_config.json',
 'final_model/special_tokens_map.json',
 'final_model/tokenizer.json')

## Delta weights

save only delta weights ( code commented as not require right now)

In [ ]:
"""
# uncomment code to extract delta weights

from peft import extract_lora_weights

# Extract the delta weights from the fine-tuned model
delta_weights = extract_lora_weights(model)

# Save the delta weights to a specified directory
delta_weights.save_pretrained("delta_weights")

"""

Later merge delta weights with original model

In [ ]:
"""
# uncomment code to merge delta weights with original model

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import apply_lora_weights

# Load the original model and tokenizer
model_id = "your-model-id"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the delta weights
delta_weights = LoraConfig.from_pretrained("delta_weights")

# Apply the delta weights to the original model
model = apply_lora_weights(model, delta_weights)

# Example usage of the model and tokenizer
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors='pt')
outputs = model.generate(inputs['input_ids'])
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

"""

In [ ]:
# wandb.finish()
model.config.use_cache = True

eval/loss,▂▁▂▂▂▂▄▂▃▅█▅▆▇▂▁▂▁▁▁▁▁▁▂▂▃▃▃▃▃▁▂▂▂▃▄▄▄▄▁
eval/runtime,▁▂▂▂▅▃▃▃▅▄▄▄▄▃▄▄▄▄▅▃▇▇█▇▇▇▇▇▇█▇▇▇▇▇▇▇█▅▆
eval/samples_per_second,█▇▄▂▆▅▅▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▂▂▁▁▁▂▂▂▂▂▁▂▁▄▄▂
eval/steps_per_second,█▅▂▇▂▆▅▅▄▆▅▅▅▆▆▅▅▆▅▅▆▅▆▂▂▂▁▂▂▂▂▁▁▂▂▂▁▂▁▄
train/epoch,▂▃█▄█▅█▄▁▂▇█▁▃▃▅█▁▂▂█▅█▁▆▄▅▇▁▃▆██▁▇▂▃█▇▆
train/global_step,▂█▁▂▃▆▄▇▅█▇▂▇██▆▇█▂▆▄▅▆█▃▇▇▁▄▆▃▄▇▂▁▂▃███
train/grad_norm,▄██▄▃ ▃▂▂▂▁▂▁▁▁▁ ▂▃▃▃ ▂▂ ▁▁▁▃▃▁▁▁▁▄ ▂▂▂
train/learning_rate,▁▁▇▆▁▆▄▃▄▆▃▃▅▂▇▄▂▂▂▅▁▅▅▄▄▄▃▂▁▁▃▅▅▆▆▄▅█▇▃
train/loss,██▇▇▆▄▇▄▇▇▅▄▃▂▂▄▃▂▁▁▁▁▁▁▇█▇█▆▄▂▂▂▁▂▁▁▁█▂
eval/loss,2.69587
eval/runtime,0.4648


# **Model Evaluation using Rouge Score**

In [ ]:
text = """user: Generate summary of this dialogue in one line
          dialogue:
          Rachel: <file_other>
          Rachel: Top 50 Best Films of 2018
          Rachel: :)
          Janice: Omg, I've watched almost all 50... xDD
          Spencer: Hahah, Deadpool 2 also??
          Janice: Yep
          Spencer: Really??
          Janice: My bf forced me to watch it xD
          Rachel: Hahah
          Janice: It wasn't that bad
          Janice: I thought it'd be worse
          Rachel: And Avengers? :D
          Janice: 2 times
          Rachel: Omg
          Janice: xP
          Rachel: You are the best gf in the world
          Rachel: Your bf should appreciate that ;-)
          Janice: He does
          Janice: x)
AI Summary:"""

device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

true_summary = "Rachel sends a list of Top 50 films of 2018. Janice watched almost half of them, Deadpool 2 and Avengers included."

outputs = model.generate(**inputs, max_new_tokens=50)
model_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(model_summary)

print("---------------------------------------------------------------------")
end_token = ""

highlight = str.strip(model_summary.split("AI Summary:")[1])
print(f"Generated Summary: {highlight}")
print("---------------------------------------------------------------------")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


user: Generate summary of this dialogue in one line
          dialogue:
          Rachel: <file_other>
          Rachel: Top 50 Best Films of 2018
          Rachel: :)
          Janice: Omg, I've watched almost all 50... xDD
          Spencer: Hahah, Deadpool 2 also??
          Janice: Yep
          Spencer: Really??
          Janice: My bf forced me to watch it xD
          Rachel: Hahah
          Janice: It wasn't that bad
          Janice: I thought it'd be worse
          Rachel: And Avengers? :D
          Janice: 2 times
          Rachel: Omg
          Janice: xP
          Rachel: You are the best gf in the world
          Rachel: Your bf should appreciate that ;-)
          Janice: He does
          Janice: x)
AI Summary: Janice and Rachel discuss their shared experience of watching the top 50 films of 2018, with Janice revealing she watched almost all of them and Spencer and Rachel also mention their favorite films from the list.
-------------------------------------------------

In [ ]:
def calculate_rouge_scores(original_summary, generated_summary):
    rouge = load_metric("rouge")
    scores = rouge.compute(
        predictions=[str.strip(generated_summary)], references=[original_summary]
    )
    return scores

In [ ]:
rouge_scores = calculate_rouge_scores(highlight, true_summary)
rouge_scorer_ = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL", "rougeLsum"])
rouge_scores = rouge_scorer_.score(highlight, true_summary)

for metric, scores in rouge_scores.items():
    print(f"{metric}:")
    print(f"Precision: {scores.precision}")
    print(f"Recall: {scores.recall}")
    print(f"F1 Score: {scores.fmeasure}")
    print()

<ipython-input-26-ed1b05a1c08b>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


rouge1:
Precision: 0.6666666666666666
Recall: 0.3888888888888889
F1 Score: 0.49122807017543857

rouge2:
Precision: 0.3
Recall: 0.17142857142857143
F1 Score: 0.21818181818181817

rougeL:
Precision: 0.6190476190476191
Recall: 0.3611111111111111
F1 Score: 0.45614035087719296

rougeLsum:
Precision: 0.6190476190476191
Recall: 0.3611111111111111
F1 Score: 0.45614035087719296



In [ ]:
rouge_scores

{'rouge1': Score(precision=0.6666666666666666, recall=0.3888888888888889, fmeasure=0.49122807017543857),
 'rouge2': Score(precision=0.3, recall=0.17142857142857143, fmeasure=0.21818181818181817),
 'rougeL': Score(precision=0.6190476190476191, recall=0.3611111111111111, fmeasure=0.45614035087719296),
 'rougeLsum': Score(precision=0.6190476190476191, recall=0.3611111111111111, fmeasure=0.45614035087719296)}

### Calculate Rouge Score on test data

In [ ]:
test_dataset = dataset_dict["validation"].select(range(5))

test_dataset = pd.DataFrame(test_dataset)

In [ ]:
num_iterations = len(test_dataset)

avg_scores = {
    "rouge1": {"precision": 0, "recall": 0, "f1": 0},
    "rouge2": {"precision": 0, "recall": 0, "f1": 0},
    "rougeL": {"precision": 0, "recall": 0, "f1": 0},
    "rougeLsum": {"precision": 0, "recall": 0, "f1": 0},
}

print("Test data matrices...")
for idx, row in test_dataset.iterrows():
    dialogue = row["dialogue"]
    true_summary = row["summary"]

    text = f"""user\n Write the highlight of this dialogue in one sentence:{dialogue}\nAI Summary:"""
    device = "cuda:0"
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=50)
    gemma_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("-" * 50)
    print(f"True Summary: {true_summary}")

    delimiter = "AI Summary:"
    end_token = ""

    highlight = str.strip(gemma_summary.split("AI Summary:")[1])
    print(f"Generated Summary: {highlight}")

    rouge_scores = calculate_rouge_scores(highlight, true_summary)
    rouge_scorer_ = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    )
    rouge_scores = rouge_scorer_.score(highlight, true_summary)

    for metric, scores in rouge_scores.items():
        rouge_scores_matrix = {
            metric: {
                "precision": scores.precision,
                "recall": scores.recall,
                "fmeasure": scores.fmeasure,
            }
        }
        # Convert the rouge_scores to a DataFrame
        df = pd.DataFrame(rouge_scores_matrix).transpose()
        print(df)

        print()
        avg_scores[metric]["precision"] += scores.precision
        avg_scores[metric]["recall"] += scores.recall
        avg_scores[metric]["f1"] += scores.fmeasure


for metric, scores in avg_scores.items():
    avg_scores[metric]["precision"] /= num_iterations
    avg_scores[metric]["recall"] /= num_iterations
    avg_scores[metric]["f1"] /= num_iterations


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


--------------------------------------------------
True Summary: A will go to the animal shelter tomorrow to get a puppy for her son. They already visited the shelter last Monday and the son chose the puppy. 
Generated Summary: A person, A, asks B, Tom, to accompany him to an animal shelter to pick up a puppy for his son. B agrees and mentions the responsibilities of raising a dog. A and B discuss the type of dog they think his son would


/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


        precision    recall  fmeasure
rouge1        0.5  0.318182  0.388889

        precision    recall  fmeasure
rouge2   0.111111  0.069767  0.085714

        precision    recall  fmeasure
rougeL   0.464286  0.295455  0.361111

           precision    recall  fmeasure
rougeLsum   0.464286  0.295455  0.361111

--------------------------------------------------
True Summary: Emma and Rob love the advent calendar. Lauren fits inside calendar various items, for instance, small toys and Christmas decorations. Her children are excited whenever they get the calendar.
Generated Summary: Emma is excited about an advent calendar and remembers them being filled with chocolates as a child. Rob shares his own experience of getting an advent calendar as a child. Lauren adds that modern advent calendars are more sophisticated and come in various forms and contents, such


/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


        precision    recall  fmeasure
rouge1   0.227273  0.113636  0.151515

        precision  recall  fmeasure
rouge2        0.0     0.0       0.0

        precision    recall  fmeasure
rougeL   0.181818  0.090909  0.121212

           precision    recall  fmeasure
rougeLsum   0.181818  0.090909  0.121212

--------------------------------------------------
True Summary: Marla found a pair of boxers under her bed.
Generated Summary: Marla finds a pair of underwear under her bed and is confused about who could have left it there. Her friends speculate that it might have been left by a former partner or a prank by her sister. They investigate the possibility that the underwear belonged to


/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


        precision    recall  fmeasure
rouge1   0.777778  0.148936      0.25

        precision    recall  fmeasure
rouge2        0.5  0.086957  0.148148

        precision    recall  fmeasure
rougeL   0.777778  0.148936      0.25

           precision    recall  fmeasure
rougeLsum   0.777778  0.148936      0.25

--------------------------------------------------
True Summary: Robert wants Fred to send him the address of the music shop as he needs to buy guitar cable.
Generated Summary: The conversation between Robert and Fred appears to be a simple exchange where Robert asks for the address of a music shop, and Fred responds by suggesting using Google Maps to find the location. Robert thanks Fred and the conversation ends with a friendly "m8"
        precision    recall  fmeasure
rouge1   0.526316  0.222222    0.3125

        precision    recall  fmeasure
rouge2   0.166667  0.068182  0.096774

        precision  recall  fmeasure
rougeL   0.473684     0.2   0.28125

           precision

/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


### Average rouge score

In [ ]:
# Convert the evaluation results to a DataFrame
df = pd.DataFrame(avg_scores)

# Transpose the DataFrame for better readability
df = df.transpose()

# Print the DataFrame
print("Test data average rouge score...")
print(df)

           precision    recall        f1
rouge1      0.482135  0.207404  0.278476
rouge2      0.162698  0.049329  0.071533
rougeL      0.434686  0.181103  0.244820
rougeLsum   0.434686  0.181103  0.244820


In [ ]:
wandb.finish()